In [2]:
%matplotlib inline

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Basic Ideas:

Spacing as a geometric series:

    dx = a * r ^ 1.05

Position is a cumulative sum of the series:

    x[k] = a * (1 - r ^ k) / (1 - r)


Reverse lookup:

    k = 1 / log(r) * log(1 - (1 - r) / a * z[k])
      = 1 / log(r) * log(1 + (r - 1) / a * z[k])

NOTE: We can use the math function `log1p()` in OpenCL

In [89]:
# Stretched grid with a = 2.0, r = 1.0212
a = 2.0
r = 1.0212

#
# RS_set_vel_data_to_LES_table() ...
# ...
# table.spacing = RSTableSpacingStretchedX | RSTableSpacingStretchedY | RSTableSpacingStretchedZ;
# table.x_ = leslie->nx;    table.xm = 0.5f * (float)(leslie->nx - 1);    table.xs = 1.0f / log(leslie->rx);    table.xo = (leslie->rx - 1.0f) / leslie->ax;
# table.y_ = leslie->ny;    table.ym = 0.5f * (float)(leslie->ny - 1);    table.ys = 1.0f / log(leslie->ry);    table.yo = (leslie->ry - 1.0f) / leslie->ay;
# table.z_ = leslie->nz;    table.zm = 0.0f;                              table.zs = 1.0f / log(leslie->rz);    table.zo = (leslie->rz - 1.0f) / leslie->az;
#
#
# RS_set_vel_data() ...
# ...
# H->workers[i].les_desc.s[RSTable3DStaggeredDescriptionBaseChangeX] = table.xs;      // "m" for stretched grid: m * log1p(n * pos.x) + o;
# H->workers[i].les_desc.s[RSTable3DStaggeredDescriptionPositionScaleX] = table.xo;   // "n" for stretched grid: m * log1p(n * pos.x) + o;
# H->workers[i].les_desc.s[RSTable3DStaggeredDescriptionOffsetX] = table.xm;          // "o" for stretched grid: m * log1p(n * pos.x) + o;
#
#
# wind_table_index() ...
# ...
# float4 pos_rel = pos - (float4)(sim_desc.hi.s01 + 0.5f * sim_desc.hi.s45, 0.0f, 0.0f);
# return copysign(wind_desc.s0123, pos_rel) * log1p(wind_desc.s4567 * fabs(pos_rel)) + wind_desc.s89ab;
#
#  k = (     m    ) * log1p((    n    ) * z[k]) + (     o      )
#  k = 1.0 / log(r) * log1p((r - 1) / a * z[k]) + (0.5 * nx - 1)
#

n = 9
m = 0.5 * (n - 1)
s = 1.0 / np.log(r)
o = (r - 1.0) / a

s0 = s
s4 = o
s8 = m

x = -10.0

k = s0 * np.sign(x) * np.log1p(s4 * np.abs(x)) + s8

# k = np.arange(n)
# x = a * (1.0 - np.power(r, k)) / (1.0 - r)
# o = a * (1.0 - np.power(r, k[-1])) / (1.0 - r)

# a = 2.0, r = 1.05

# m * log1p(n * pos.x) + o;

# x = np.arange(0, nx + 1) * dx - 1.0
# y = np.arange(0, ny + 1) * dy - 1.0

# # Generate a desired PDF & CDF
# xx, yy  = np.meshgrid(x[:-1], y[:-1])
# # f = np.exp(-(xx - 0.2) ** 2 / 0.2 + 0.1 * (xx + 0.1) * (yy - 0.1) / 0.02 - (yy + 0.2) ** 2 / 0.1)
# f = np.exp(-(xx - 0.4) ** 2 / 0.2 - 0.1 * (xx - 0.1) * (yy + 0.15) / 0.02 - (yy + 0.2) ** 2 / 0.1)
# pdf = f / f.sum()
# cdf = pdf.cumsum()

# xe = np.arange(0, nx + 1) * dx - 1.0
# ye = np.arange(0, ny + 1) * dy - 1.0

In [90]:
# plt.plot(np.diff(x), 'o')
# x = np.arange(-20, 20, 1.0)
# k = np.sign(x) * a * np.log1p(r * abs(x)) + 

In [91]:
np.set_printoptions(formatter={'float': ' {:6.3f}'.format})

In [92]:
k = np.arange(0, m + 1)
x = a * (1.0 - np.power(r, k)) / (1.0 - r)
dx = np.concatenate(([0], a * np.power(r, k[:-1])))

print('         k = {}'.format(np.array(k, dtype=float)))
print('        dx = {}'.format(dx))
print('cumsum(dx) = {}'.format(np.cumsum(dx)))
print('         x = {}'.format(x))

         k = [  0.000   1.000   2.000   3.000   4.000]
        dx = [  0.000   2.000   2.042   2.086   2.130]
cumsum(dx) = [  0.000   2.000   4.042   6.128   8.258]
         x = [ -0.000   2.000   4.042   6.128   8.258]


In [93]:
k = np.array(np.arange(-m, m+1), dtype=float)
i = np.array(np.arange(len(k)), dtype=float)
x = np.sign(k) * a * (1.0 - np.power(r, np.abs(k))) / (1.0 - r)

# Compute the table index
k0 = s0 * np.sign(x) * np.log1p(s4 * np.abs(x)) + s8

In [95]:
print(' i = {}'.format(i))
print(' k = {}'.format(k))
print(' x = {}'.format(x))
print('')
print('k0 = {}'.format(k0))

 i = [  0.000   1.000   2.000   3.000   4.000   5.000   6.000   7.000   8.000]
 k = [ -4.000  -3.000  -2.000  -1.000   0.000   1.000   2.000   3.000   4.000]
 x = [ -8.258  -6.128  -4.042  -2.000  -0.000   2.000   4.042   6.128   8.258]

k0 = [ -0.000   1.000   2.000   3.000   4.000   5.000   6.000   7.000   8.000]
